In [1]:
%load_ext autoreload
%autoreload 2

from fastai.text.all import *
from fastai.vision.all import *
import pandas as pd
import torch
from tqdm.notebook import tqdm

from utils import get_dls

In [2]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

# # tensorflow RNG
# tf.random.set_seed(seed)

In [3]:
path = Path("/mnt/nas/backups/08-07-2020/desktopg01/lisa/Data/small_flow")

In [20]:
dls = get_dls(path, 64, 224)

In [21]:
test_items = get_image_files(path, folders="test")

In [22]:
test_dl = dls.test_dl(test_items, with_labels=True)

In [190]:
class GetActs(HookCallback):
    def __init__(self, modules=None, remove_end=True, detach=True, cpu=False):
        super().__init__(modules, None, remove_end, True, detach, cpu)
        self.acts = L()
    def hook(self, m, i, o): return o
    def after_pred(self): self.acts += self.hooks.stored
    def before_fit(self):
        super().before_fit()
        self.acts = L()

In [191]:
learn = cnn_learner(dls, resnet50, loss_func=CrossEntropyLossFlat())

In [192]:
learn.add_cb(GetActs([learn.model[1][1]]))

In [ ]:
learn.load("resnet50-fine-tuned-1E-disc-6E-224_class_weights")

In [ ]:
learn.validate()

In [ ]:
learn.get_acts.acts

In [ ]:
valid_preds = []
with hook_output(learn.model[1][1]) as hook:
    for b in tqdm(dls.valid):
        learn.model(b[0])
        valid_preds += hook.stored
valid_preds

In [ ]:
torch.stack(valid_preds).shape